# 6 - Enrutamiento por similitud semántica

<img src="https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/langchain.jpeg" style="width:400px;"/>

<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1---Enrutamiento-por-similitud-semántica" data-toc-modified-id="1---Enrutamiento-por-similitud-semántica-1">1 - Enrutamiento por similitud semántica</a></span></li></ul></div>

## 1 - Enrutamiento por similitud semántica

LCEL nos permite implementar lógica de enrutamiento personalizada basada en la similitud semántica de la entrada del usuario. Veamos un ejemplo de cómo determinar dinámicamente la lógica de la cadena según la entrada del usuario.

In [1]:
# cargamos la API KEY de OpenAI

from dotenv import load_dotenv 
import os

# carga de variables de entorno
load_dotenv()


# api key openai, nombre que tiene por defecto en LangChain
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [2]:
# importamos librerias

from langchain_openai import ChatOpenAI

from langchain_openai import OpenAIEmbeddings

from langchain.prompts import PromptTemplate

from langchain.schema.output_parser import StrOutputParser

from langchain.schema.runnable import RunnableLambda

from langchain.utils.math import cosine_similarity

In [3]:
# iniciamos el modelo de OpenAI

modelo = ChatOpenAI()

In [4]:
# parser de salida, transforma la salida a string

parser = StrOutputParser()

In [5]:
# definimos el primer prompt

plantilla_fisica = '''Eres un profesor de física muy inteligente.
                      Eres excelente respondiendo preguntas sobre física de 
                      manera concisa y fácil de entender.
                      Cuando no sabes la respuesta a una pregunta, admites que no la sabes.
                      
                      Aquí tienes una pregunta:
                      {pregunta}
                      
                    '''

In [6]:
# definimos el segundo prompt

plantilla_matematicas = '''Eres un muy buen matemático. 
                           Eres excelente respondiendo preguntas de matemáticas.
                           Eres tan bueno porque puedes descomponer problemas difíciles 
                           en sus partes componentes, responder esas partes y 
                           luego juntarlas para responder la pregunta más amplia.
                           
                           Aquí tienes una pregunta:
                           {pregunta}
                        '''

In [7]:
# los ponemos en una lista para usarlos despues

plantillas = [plantilla_fisica, plantilla_matematicas]

In [8]:
# iniciamos el modelo de embedding

modelo_embeddings = OpenAIEmbeddings()

In [9]:
# realizamos la vectorizacion de los prompts

prompt_embeddings = modelo_embeddings.embed_documents(plantillas)

In [10]:
# longitud del vector de embedding

len(prompt_embeddings[0])

1536

In [11]:
# funcion para enrutar el prompt

def enrutador_prompt(usuario: dict) -> PromptTemplate.from_template:
    
    """
    Esta función decide el comportamiento del modelo segun 
    la pregunta del usuario. Compara la similitud entre la pregunta realizada
    y los prompts de matematicas o fisica y decide cual de ellos usar.
    
    Params:
    usuario: dict, diccionario con la key 'pregunta'
    
    Return:
    objeto PromptTemplate.from_template para ser pasado a la cadena
    """
    
    global modelo_embeddings, prompt_embeddings, plantillas

    embedding_pregunta = modelo_embeddings.embed_query(usuario['pregunta'])
    
    similitud = cosine_similarity([embedding_pregunta], prompt_embeddings)[0]
    
    mas_similar = plantillas[similitud.argmax()]
    
    print('Usando MATEMATICAS' if mas_similar == plantilla_matematicas else 'Usando FISICA')
    
    return PromptTemplate.from_template(mas_similar)

In [12]:
# creamos la cadena

cadena = RunnableLambda(enrutador_prompt) | modelo | parser

In [13]:
# respuesta de la cadena

cadena.invoke({'pregunta': 'qué es un agujero negro'})

Usando FISICA


'Un agujero negro es una región del espacio donde la gravedad es tan intensa que nada, ni siquiera la luz, puede escapar de su atracción. Se forma cuando una estrella masiva colapsa sobre sí misma y su núcleo se comprime tanto que crea un campo gravitacional extremadamente fuerte. Todo lo que cae dentro de un agujero negro es absorbido por él, creando una región de la cual nada puede salir. Los agujeros negros son objetos fascinantes y aún hay mucho por descubrir sobre ellos en el campo de la física.'

In [14]:
# respuesta de la cadena

cadena.invoke({'pregunta': 'qué es una integral de caminos'})

Usando MATEMATICAS


'Una integral de caminos es un concepto utilizado en el campo de la teoría de la probabilidad y en la física cuántica. Se refiere a la integral de una función a lo largo de todos los posibles caminos o trayectorias que puede seguir una partícula desde un punto A hasta un punto B en un espacio determinado. En lugar de calcular la integral a lo largo de una única trayectoria específica, se consideran todas las posibles trayectorias y se suman sus contribuciones. Este enfoque se utiliza en situaciones donde el comportamiento de una partícula no puede ser descrito de manera determinista, sino que se rige por la probabilidad. La integral de caminos es una herramienta matemática poderosa que se utiliza para modelar sistemas complejos y estudiar fenómenos cuánticos.'